In [ ]:
import numpy as np
import tensorflow as tf
import keras
from keras import layers, Sequential, Input
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import plot_model

import glob
import os
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
n_classes = 2
input_shape=(256, 256, 1)

In [ ]:
class UNet():
    def __init__(self):

        #Encoder
        self.block1_conv1 = layers.Conv2D(filters=64, kernel_size=(3, 3), name='block1_conv1', activation='relu', padding='same')
        self.block1_conv2 = layers.Conv2D(filters=64, kernel_size=(3, 3), name='block1_conv2', padding='same')
        self.block1_bn = layers.BatchNormalization()
        self.block1_act = layers.ReLU()
        self.block1_pool = layers.MaxPooling2D(pool_size=(2, 2), strides=None, name='block1_pool')

        self.block2_conv1 = layers.Conv2D(filters=128, kernel_size=(3, 3), name='block2_conv1', activation='relu', padding='same')
        self.block2_conv2 = layers.Conv2D(filters=128, kernel_size=(3, 3), name='block2_conv2', padding='same')
        self.block2_bn = layers.BatchNormalization()
        self.block2_act = layers.ReLU()
        self.block2_pool = layers.MaxPooling2D(pool_size=(2, 2), strides=None, name='block2_pool')

        self.block3_conv1 = layers.Conv2D(filters=256, kernel_size=(3, 3), name='block3_conv1', activation='relu', padding='same')
        self.block3_conv2 = layers.Conv2D(filters=256, kernel_size=(3, 3), name='block3_conv2', padding='same')
        self.block3_bn = layers.BatchNormalization()
        self.block3_act = layers.ReLU()
        self.block3_pool = layers.MaxPooling2D(pool_size=(2, 2), strides=None, name='block3_pool')

        self.block4_conv1 = layers.Conv2D(filters=512, kernel_size=(3, 3), name='block4_conv1', activation='relu', padding='same')
        self.block4_conv2 = layers.Conv2D(filters=512, kernel_size=(3, 3), name='block4_conv2', padding='same')
        self.block4_bn = layers.BatchNormalization()
        self.block4_act = layers.ReLU()
        self.block4_dropout = layers.Dropout(0.5)
        self.block4_pool = layers.MaxPooling2D(pool_size=(2, 2), strides=None, name='block4_pool')

        self.block5_conv1 = layers.Conv2D(filters=1024, kernel_size=(3, 3), name='block5_conv1', activation='relu', padding='same')
        self.block5_conv2 = layers.Conv2D(filters=1024, kernel_size=(3, 3), name='block5_conv2', padding='same')
        self.block5_bn = layers.BatchNormalization()
        self.block5_act = layers.ReLU()
        self.block5_dropout = layers.Dropout(0.5)

        #Decoder
        self.block6_up = tf.keras.layers.UpSampling2D(size = (2,2))
        self.block6_conv1 = tf.keras.layers.Conv2D(512, (2, 2) , name='block6_conv1', activation = 'relu', padding = 'same')
        self.block6_conv2 = tf.keras.layers.Conv2D(512, (3, 3) , name='block6_conv2', activation = 'relu', padding = 'same')
        self.block6_conv3 = tf.keras.layers.Conv2D(512, (3, 3) , name='block6_conv3', padding = 'same')
        self.block6_bn = tf.keras.layers.BatchNormalization()
        self.block6_act = tf.keras.layers.ReLU()

        self.block7_up = tf.keras.layers.UpSampling2D(size = (2,2))
        self.block7_conv1 = tf.keras.layers.Conv2D(256, (2, 2) , name='block7_conv1', activation = 'relu', padding = 'same')
        self.block7_conv2 = tf.keras.layers.Conv2D(256, (3, 3) , name='block7_conv2', activation = 'relu', padding = 'same')
        self.block7_conv3 = tf.keras.layers.Conv2D(256, (3, 3) , name='block7_conv3', padding = 'same')
        self.block7_bn = tf.keras.layers.BatchNormalization()
        self.block7_act = tf.keras.layers.ReLU()

        self.block8_up = tf.keras.layers.UpSampling2D(size = (2,2))
        self.block8_conv1 = tf.keras.layers.Conv2D(128, (2, 2) , name='block8_conv1', activation = 'relu', padding = 'same')
        self.block8_conv2 = tf.keras.layers.Conv2D(128, (3, 3) , name='block8_conv2', activation = 'relu', padding = 'same')
        self.block8_conv3 = tf.keras.layers.Conv2D(128, (3, 3) , name='block8_conv3', padding = 'same')
        self.block8_bn = tf.keras.layers.BatchNormalization()
        self.block8_act = tf.keras.layers.ReLU()

        self.block9_up = tf.keras.layers.UpSampling2D(size = (2,2))
        self.block9_conv1 = tf.keras.layers.Conv2D(64, (2, 2) , name='block9_conv1', activation = 'relu', padding = 'same')
        self.block9_conv2 = tf.keras.layers.Conv2D(64, (3, 3) , name='block9_conv2', activation = 'relu', padding = 'same')
        self.block9_conv3 = tf.keras.layers.Conv2D(64, (3, 3) , name='block9_conv3', padding = 'same')
        self.block9_bn = tf.keras.layers.BatchNormalization()
        self.block9_act = tf.keras.layers.ReLU()

        self.output_conv = tf.keras.layers.Conv2D(n_classes, (1, 1), name='output_conv', activation = 'sigmoid')

    def model(self):

        inputs = Input(shape=(input_shape))

        x1 = self.block1_conv1(inputs)
        x1 = self.block1_conv2(x1)
        x1 = self.block1_bn(x1)
        x1_conc = self.block1_act(x1)
        x1_out = self.block1_pool(x1_conc)

        x2 = self.block2_conv1(x1_out)
        x2 = self.block2_conv2(x2)
        x2 = self.block2_bn(x2)
        x2_conc = self.block2_act(x2)
        x2_out = self.block2_pool(x2_conc)

        x3 = self.block3_conv1(x2_out)
        x3 = self.block3_conv2(x3)
        x3 = self.block3_bn(x3)
        x3_conc = self.block3_act(x3)
        x3_out = self.block3_pool(x3_conc)

        x4 = self.block4_conv1(x3_out)
        x4 = self.block4_conv2(x4)
        x4 = self.block4_bn(x4)
        x4 = self.block4_act(x4)
        x4_conc = self.block4_dropout(x4)
        x4_out = self.block4_pool(x4_conc)

        x5 = self.block5_conv1(x4_out)
        x5 = self.block5_conv2(x5)
        x5 = self.block5_bn(x5)
        x5 = self.block5_act(x5)
        x5_out = self.block5_dropout(x5)

        x6 = self.block6_up(x5_out)
        x6 = layers.Concatenate(axis=3)([x6, x4_conc])
        x6 = self.block6_conv1(x6)
        x6 = self.block6_conv2(x6)
        x6 = self.block6_conv3(x6)
        x6 = self.block6_bn(x6)
        x6_out = self.block6_act(x6)

        x7 = self.block7_up(x6_out)
        x7 = layers.Concatenate(axis=3)([x7, x3_conc])
        x7 = self.block7_conv1(x7)
        x7 = self.block7_conv2(x7)
        x7 = self.block7_conv3(x7)
        x7 = self.block7_bn(x7)
        x7_out = self.block7_act(x7)

        x8 = self.block8_up(x7_out)
        x8 = layers.Concatenate(axis=3)([x8, x2_conc])
        x8 = self.block8_conv1(x8)
        x8 = self.block8_conv2(x8)
        x8 = self.block8_conv3(x8)
        x8 = self.block8_bn(x8)
        x8_out = self.block8_act(x8)

        x9 = self.block9_up(x8_out)
        x9 = layers.Concatenate(axis=3)([x9, x1_conc])
        x9 = self.block9_conv1(x9)
        x9 = self.block9_conv2(x9)
        x9 = self.block9_conv3(x9)
        x9 = self.block9_bn(x9)
        x9_out = self.block9_act(x9)

        outputs = self.output_conv(x9_out)

        model = Model(inputs=inputs, outputs=outputs)


        return model